In [2]:
import pandas as pd
from collections import Counter

In [11]:
df = pd.read_csv('data/merged_data.csv')

In [4]:
df.shape

(75059, 9)

TypeError: eval() arg 1 must be a string, bytes or code object

In [5]:
def get_state(answer, guess):
    count_letters = Counter(answer)
    state = [0] * 5
    # print(count_letters)

    # This marks the greens
    for index, letter in enumerate(guess):
        # print(letter)
        if letter in count_letters:
            # print(f'letter: {letter}')
            if letter == answer[index] and count_letters[letter] > 0:
                state[index] = 2
                count_letters[letter] -= 1

    # This marks the yellows
    for index, letter in enumerate(guess):
        if letter in count_letters:
            if count_letters[letter] > 0 and state[index] == 0:
                state[index] = 1
                count_letters[letter] -= 1

    subset = [0] * 3

    for s in state:
        subset[2-s] += 1

    return "".join(map(str, subset))

In [6]:
answer = 'banal'
guess = 'daddy'

print(get_state(answer, guess))

104


In [117]:
for index, row in df.iterrows():
    # drop the row if the guess is not 5 letters
    if len(eval(row['wordle_guesses'])) != len(eval(row['optimal'])) + 1:
        df.drop(index, inplace=True)

def check_if_guess_duplicate_as_answer(df):
    for idx, row in df.iterrows():
        if row['wordle_guesses'].count(row['wordle_answer']) != 1:
            # delete current row
            # print(row['entry_id'])
            df.drop(idx, inplace=True)

check_if_guess_duplicate_as_answer(df)


18
58
74
103
115
158
166
170
234
280
307
457
523
525
592
709
717
744
816
912
1011
1053
1120
1125
1166
1201
1211
1215
1222
1310
1322
1360
1364
1406
1421
1505
1520
1523
1558
1622
1643
1649
1684
1688
1739
1826
1857
1859
1907
1908
1929
1937
1963
2055
2080
2104
2154
2215
2216
2276
2352
2672
2719
2747
2847
2887
2919
2925
3022
3031
3199
3212
3231
3246
3253
3288
3319
3326
3422
3451
3472
3478
3480
3482
3485
3496
3501
3511
3554
3612
3635
3777
3886
3955
4012
4047
4281
4910
4980
5026
5227
5254
5324
5617
5632
5727
5940
5966
6039
6254
6367
7570
7699
8141
8148
8175
8241
8414
8610
8632
8683
8891
9199
9333
9342
9522
10514
11156
11527
11888
12060
12212
12437
12663
12716
13325
13419
13831
14150
14484
14524
14839
15204
15426
15544
15605
16017
16190
16701
16706
17048
17359
17652
17684
17802
17893
17910
18048
18250
18582
18792
18794
18877
19072
19374
19604
19886
20237
20412
20457
20562
20788
21003
21347
21998
22384
22409
22865
23108
23403
23631
23943
24223
24535
24865
25208
25534
25770
26028
26358
26639
269

In [12]:
df.shape

(75059, 9)

In [14]:
# make a new data frame called state_data where we only keep the optimal and answer columns
data_needed = df[['optimal', 'wordle_answer', 'wordle_guesses']].copy()
data_needed.head()

,optimal,wordle_answer,wordle_guesses
0,"[[('WORLD', 244), ('SAICE', 8)], [('LEAFS', 21...",banal,"['world', 'leafs', 'clang', 'bantu', 'banal']"
1,"[[('CRIES', 256), ('TONAL', 1)], [('DADDY', 30...",banal,"['cries', 'daddy', 'water', 'manor', 'bangs', ..."
2,"[[('MOUSE', 327), ('CLINT', 3)], [('TRAIL', 5)...",banal,"['mouse', 'trail', 'halal', 'papal', 'canal', ..."
3,"[[('AROSE', 148), ('CLINT', 4)], [('TAUNT', 16...",banal,"['arose', 'taunt', 'naval', 'banal']"
4,"[[('STALE', 57), ('LORAL', 8)], [('ALONG', 7),...",banal,"['stale', 'along', 'final', 'canal', 'banal']"


In [50]:
# make a test dataframe that is the first 10 rows of the data_needed dataframe
test_data = data_needed.head(10)

In [15]:
# specify the data types
# still some issues with "['later', 'humor', 'salet', 'spill']",4,humor
from tqdm import tqdm
state_data_final = pd.DataFrame(columns=['state', 'previous_guesses', 'optimal_guess', 'human_guess'])

previous_dict = {}
optimal_dict = {}
human_dict = {}

# iterate over the rows of the data frame
for index, row in tqdm(data_needed.iterrows(), total=data_needed.shape[0]):
    answer = row['wordle_answer']
    guesses = eval(row['wordle_guesses'])
    optimal = eval(row['optimal'])

    for i in range(len(guesses) - 1):
        state = get_state(answer, guesses[i])
        try:
            opt_guess = optimal[i][1][0]
        except:
            print(index)
            print(optimal)
            print(i)
            print(guesses)
        # print(opt_guess)
        # check if the state is already in the state column
        if state not in previous_dict:
            previous_dict[state] = []
            optimal_dict[state] = []
            human_dict[state] = []

        previous_dict[state].append(guesses[:i+1])
        optimal_dict[state].append(opt_guess)
        human_dict[state].append(guesses[i+1])
        
for state in previous_dict:
    state_data_final = state_data_final.append({'state': state, 'previous_guesses': previous_dict[state], 'optimal_guess': optimal_dict[state], 'human_guess': human_dict[state]}, ignore_index=True)

# state_data_final.to_csv('state_data.csv', index=False)
        





100%|██████████| 75059/75059 [00:13<00:00, 5582.57it/s]
C:\Users\garyl\AppData\Local\Temp\ipykernel_21092\90825072.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  state_data_final = state_data_final.append({'state': state, 'previous_guesses': previous_dict[state], 'optimal_guess': optimal_dict[state], 'human_guess': human_dict[state]}, ignore_index=True)
C:\Users\garyl\AppData\Local\Temp\ipykernel_21092\90825072.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  state_data_final = state_data_final.append({'state': state, 'previous_guesses': previous_dict[state], 'optimal_guess': optimal_dict[state], 'human_guess': human_dict[state]}, ignore_index=True)
C:\Users\garyl\AppData\Local\Temp\ipykernel_21092\90825072.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

In [16]:
state_data_final.to_csv('data/state_data.csv', index=False)